Copyright (c) Microsoft Corporation. All rights reserved.

Licensed under the MIT License.

![Impressions](https://PixelServer20190423114238.azurewebsites.net/api/impressions/MachineLearningNotebooks/how-to-use-azureml/ml-frameworks/pytorch/distributed-pytorch-with-horovod/distributed-pytorch-with-horovod.png)

# Distributed PyTorch with DistributedDataParallel
In this tutorial, you will train a PyTorch model on the [CIFAR10](http://www.cs.toronto.edu/~kriz/cifar.html) dataset using distributed training with PyTorch's `DistributedDataParallel` module across a Azure Stack Hub CPU Kubernetes cluster.

## Prerequisites

Please see Prerequisite part of [this notebook](../pipeline/nyc-taxi-data-regression-model-building.ipynb)

In [1]:
# Check core SDK version number
import azureml.core

print("SDK version:", azureml.core.VERSION)

C:\Users\v-songshanli\Anaconda3\envs\pythonProject\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\v-songshanli\Anaconda3\envs\pythonProject\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\v-songshanli\Anaconda3\envs\pythonProject\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:\n%s" %
Failure while loading azureml_run_type_providers. Failed to load entrypoint hyperdrive = azureml.train.hyperdrive:HyperDriveRun._from_run_dto with exception (azureml-telemetry 1.19.0 (c:\users\v-songshanli\anaconda3\envs\pythonproject\lib\site-packages), Requirement.parse('azureml-telemetry~=1.18.0')).
Failure while loading azureml_run_type_providers. Failed to load entrypoint automl = azureml.train.automl.run:AutoMLRun._from_run_dto with exception (azureml-core 1.19.0 (c:\users\v-songshanl

SDK version: 1.19.0


## Diagnostics
Opt-in diagnostics for better experience, quality, and security of future releases.

In [2]:
from azureml.telemetry import set_diagnostics_collection

set_diagnostics_collection(send_diagnostics=True)

Turning diagnostics collection on. 


## Initialize workspace

Initialize a [Workspace](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#workspace) object from the existing workspace you created in the Prerequisites step. `Workspace.from_config()` creates a workspace object from the details stored in `config.json`. 

If you haven't done already please go to `config.json` file and fill in your workspace information.

In [3]:
from azureml.core.workspace import Workspace,  ComputeTarget
from azureml.exceptions import ComputeTargetException

ws = Workspace.from_config()
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep='\n')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


Workspace name: sl-ash2-mal
Azure region: eastus
Subscription id: 6b736da6-3246-44dd-a0b8-b5e95484633d
Resource group: sl-ash2


## Prepare dataset

Here we download cifar10 dataset from [cifar10-data](https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz). The downloaded data is then registered as dataset in a data store of the workspace. 

To set up datastore using an azure stack hub storage account, please refer to [Train_azure_arc](https://github.com/Azure/AML-Kubernetes/blob/master/docs/ASH/Train-AzureArc.md#create-and-configure-azure-stack-hubs-storage-account). 

To register the dataset manually, please refer to this [video](https://msit.microsoftstream.com/video/51f7a3ff-0400-b9eb-2703-f1eb38bc6232)


In [4]:
import tempfile
import os
import requests
from azureml.core import Dataset

from azureml.exceptions import UserErrorException
dataset_name = 'CIFAR-10'
datastore_name = "ashstore"

if dataset_name not  in ws.datasets:
    path = 'https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz'

    with tempfile.TemporaryDirectory() as tmpdir:
        os.makedirs(os.path.join(tmpdir, 'cifar-10'))

        data = requests.get(path, allow_redirects=True).content
        with open(os.path.join(tmpdir, 'cifar-10', path.split('/')[-1]), 'wb') as f:
            f.write(data)
    
        
        ds = Dataset.File.upload_directory(tmpdir, ws.datastores.get(datastore_name), overwrite=True)
        ds.register(ws, dataset_name, 'CIFAR-10 images from https://www.cs.toronto.edu/~kriz/cifar.html')
        
dataset_ash = ws.datasets[dataset_name]

## Create or attach existing ArcKubernetesCompute

The attaching code here depends  python package azureml-contrib-k8s which current is in private preview. Install private preview branch of AzureML SDK by running following command (private preview):

<pre>
pip install --disable-pip-version-check --extra-index-url https://azuremlsdktestpypi.azureedge.net/azureml-contrib-k8s-preview/D58E86006C65 azureml-contrib-k8s
</pre>

In [5]:
from azureml.contrib.core.compute.arckubernetescompute import ArcKubernetesCompute

resource_id = "/subscriptions/6b736da6-3246-44dd-a0b8-b5e95484633d/resourceGroups/AML-stack-val/providers/Microsoft.Kubernetes/connectedClusters/kub-orlando-Test"

attach_config = ArcKubernetesCompute.attach_configuration(
    resource_id= resource_id,
)

try:
    attach_name = "peymanarc"
    arcK_target_result = ArcKubernetesCompute.attach(ws, attach_name, attach_config)
    arcK_target_result.wait_for_completion(show_output=True)
    print('arc attach  success')
except ComputeTargetException as e:
    print(e)
    print('arc attach  failed')

attach_name = "nc6"
arcK_target = ws.compute_targets[attach_name]

SucceededProvisioning operation finished, operation "Succeeded"
arc attach  success


## Train model on the remote compute
Now that we have the ArcKubernetesCompute ready to go, let's run our distributed training job.

### Create a project directory
Create a directory that will contain all the necessary code from your local machine that you will need access to on the remote resource. This includes the training script and any additional files your training script depends on.

In [6]:
import os

project_folder = './pytorch-distr'
os.makedirs(project_folder, exist_ok=True)

### Prepare training script
Now you will need to create your training script. In this tutorial, the script for distributed training of CIFAR10 is already provided for you at `cifar_dist_main.py`. In practice, you should be able to take any custom PyTorch training script as is and run it with Azure ML without having to modify your code.

Once your script is ready, copy the training script `cifar_dist_main.py` into the project directory.

In [7]:
import shutil

run_script = 'cifar_dist_main.py'
shutil.copy(run_script, project_folder)

'./pytorch-distr\\cifar_dist_main.py'

### Create an experiment
Create an [Experiment](https://docs.microsoft.com/azure/machine-learning/service/concept-azure-machine-learning-architecture#experiment) to track all the runs in your workspace for this distributed PyTorch tutorial. 

In [8]:
from azureml.core import Experiment

experiment_name = 'pytorch-cifar-distr'
experiment = Experiment(ws, name=experiment_name)

### Create an environment

Define a conda environment YAML file with your training script dependencies and create an Azure ML environment.

In [9]:
%%writefile conda_dependencies.yml

channels:
- conda-forge
dependencies:
- python=3.6.2
- pip:
  - azureml-defaults
  - torch==1.6.0
  - torchvision==0.7.0
  - future==0.17.1

Overwriting conda_dependencies.yml


In [10]:
from azureml.core import Environment

pytorch_env = Environment.from_conda_specification(name = 'pytorch-1.6-cpu', file_path = './conda_dependencies.yml')

# Specify a CPU base image

pytorch_env.docker.enabled = True
pytorch_env.docker.base_image = 'mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04'


### Configure the training job: torch.distributed with GLOO backend

Create a ScriptRunConfig object to specify the configuration details of your training job, including your training script, environment to use, and the compute target to run on.

In order to run a distributed PyTorch job with **torch.distributed** using the GLOO backend, create a `PyTorchConfiguration` and pass it to the `distributed_job_config` parameter of the ScriptRunConfig constructor. Specify `communication_backend='Gloo'` in the PyTorchConfiguration. The below code will configure node_count = 2. These is the number of worker nodes. The number of  distributed jobs will be 3 if one master node is used.  GLOO backend which is recommended backend for communications between CPUs.

In [11]:
from azureml.core import ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core import Dataset
import os

dataset_ash = Dataset.get_by_name(ws, name=dataset_name)
args = [
        '--data-folder', dataset_ash.as_mount(),
        '--dist-backend', 'gloo',
           ]

distributed_job_config=PyTorchConfiguration(communication_backend='Gloo', node_count=1) #configuring AML pytorch config


src = ScriptRunConfig(
                     source_directory=project_folder,
                      script=run_script,
                      arguments=args,
                      compute_target=arcK_target,
                      environment=pytorch_env,
                      distributed_job_config=distributed_job_config)

### Submit job
Run your experiment by submitting your ScriptRunConfig object. Note that this call is asynchronous.

In [12]:
run = experiment.submit(src)
run.wait_for_completion(show_output=True) # this provides a verbose log

RunId: pytorch-cifar-distr_1612374460_efcce926
Web View: https://ml.azure.com/experiments/pytorch-cifar-distr/runs/pytorch-cifar-distr_1612374460_efcce926?wsid=/subscriptions/6b736da6-3246-44dd-a0b8-b5e95484633d/resourcegroups/sl-ash2/workspaces/sl-ash2-mal

Streaming azureml-logs/55_azureml-execution-tvmps_bedc8b36f9ae9719c550b610814f7273b73a73cd5080aabc7c698d358ae2a61d_d.txt

2021-02-03T17:48:06Z Starting output-watcher...
2021-02-03T17:48:06Z IsDedicatedCompute == True, won't poll for Low Pri Preemption
2021-02-03T17:48:06Z Executing 'Copy ACR Details file' on 10.0.0.4
2021-02-03T17:48:07Z Copy ACR Details file succeeded on 10.0.0.4. Output: 
>>>   
>>>   
Login Succeeded
Using default tag: latest
latest: Pulling from azureml/azureml_c6f9450edc622673b737209ed4accfa0
Digest: sha256:9f25d7590e2707af36af8a89a24d7ce4afda596fbe12058223bb1ffdee6dcaa2
Status: Image is up to date for viennaglobal.azurecr.io/azureml/azureml_c6f9450edc622673b737209ed4accfa0:latest
viennaglobal.azurecr.io/azur


Streaming azureml-logs/70_driver_log_0.txt

bash: /azureml-envs/azureml_77ae6faafb422d20b955420f1d57d91e/lib/libtinfo.so.5: no version information available (required by bash)
[2021-02-03T17:49:14.592674] Entering context manager injector.
[context_manager_injector.py] Command line Options: Namespace(inject=['ProjectPythonPath:context_managers.ProjectPythonPath', 'Dataset:context_managers.Datasets', 'RunHistory:context_managers.RunHistory', 'TrackUserError:context_managers.TrackUserError', 'UserExceptions:context_managers.UserExceptions'], invocation=['cifar_dist_main.py', '--data-folder', 'DatasetConsumptionConfig:input__6228182c', '--dist-backend', 'gloo'])
This is a PyTorch job. Rank:0
Script type = None
Starting the daemon thread to refresh tokens in background for process with pid = 107
Entering Run History Context Manager.
[2021-02-03T17:49:19.052341] Current directory: /mnt/batch/tasks/shared/LS_root/jobs/sl-ash2-mal/azureml/pytorch-cifar-distr_1612374460_efcce926/mounts/worksp

206 391 Loss: 1.866 | Acc: 30.057% (7964/26496)
207 391 Loss: 1.866 | Acc: 30.067% (8005/26624)
208 391 Loss: 1.864 | Acc: 30.144% (8064/26752)
209 391 Loss: 1.863 | Acc: 30.164% (8108/26880)
210 391 Loss: 1.862 | Acc: 30.210% (8159/27008)
211 391 Loss: 1.862 | Acc: 30.233% (8204/27136)
212 391 Loss: 1.860 | Acc: 30.260% (8250/27264)
213 391 Loss: 1.860 | Acc: 30.286% (8296/27392)
214 391 Loss: 1.860 | Acc: 30.316% (8343/27520)
215 391 Loss: 1.859 | Acc: 30.328% (8385/27648)
216 391 Loss: 1.858 | Acc: 30.357% (8432/27776)
217 391 Loss: 1.857 | Acc: 30.404% (8484/27904)
218 391 Loss: 1.856 | Acc: 30.458% (8538/28032)
219 391 Loss: 1.855 | Acc: 30.515% (8593/28160)
220 391 Loss: 1.854 | Acc: 30.504% (8629/28288)
221 391 Loss: 1.853 | Acc: 30.536% (8677/28416)
222 391 Loss: 1.852 | Acc: 30.542% (8718/28544)
223 391 Loss: 1.852 | Acc: 30.542% (8757/28672)
224 391 Loss: 1.851 | Acc: 30.552% (8799/28800)
225 391 Loss: 1.850 | Acc: 30.562% (8841/28928)
226 391 Loss: 1.849 | Acc: 30.575% (8884


Streaming azureml-logs/75_job_post-tvmps_bedc8b36f9ae9719c550b610814f7273b73a73cd5080aabc7c698d358ae2a61d_d.txt

[2021-02-03T17:54:04.983018] Entering job release
[2021-02-03T17:54:06.212490] Starting job release
[2021-02-03T17:54:06.212996] Logging experiment finalizing status in history service.
[2021-02-03T17:54:06.213139] job release stage : upload_datastore starting...
Starting the daemon thread to refresh tokens in background for process with pid = 450[2021-02-03T17:54:06.213431] job release stage : start importing azureml.history._tracking in run_history_release.

[2021-02-03T17:54:06.213730] job release stage : execute_job_release starting...[2021-02-03T17:54:06.213818] job release stage : copy_batchai_cached_logs starting...
[2021-02-03T17:54:06.213871] job release stage : copy_batchai_cached_logs completed...

[2021-02-03T17:54:06.267485] Entering context manager injector.
[2021-02-03T17:54:06.327248] job release stage : upload_datastore completed...
[2021-02-03T17:54:06.454

{'runId': 'pytorch-cifar-distr_1612374460_efcce926',
 'target': 'nc6',
 'status': 'Completed',
 'startTimeUtc': '2021-02-03T17:48:03.513969Z',
 'endTimeUtc': '2021-02-03T17:54:21.706679Z',
 'properties': {'_azureml.ComputeTargetType': 'amlcompute',
  'ContentSnapshotId': '453c2296-3283-4852-a71e-b46450140f6b',
  'azureml.git.repository_uri': 'git@github.com:lisongshan007/AML-Kubernetes.git',
  'mlflow.source.git.repoURL': 'git@github.com:lisongshan007/AML-Kubernetes.git',
  'azureml.git.branch': 'master',
  'mlflow.source.git.branch': 'master',
  'azureml.git.commit': '36f2bc7e95e9198cb21a83f95d15ae28daa7c93f',
  'mlflow.source.git.commit': '36f2bc7e95e9198cb21a83f95d15ae28daa7c93f',
  'azureml.git.dirty': 'True',
  'ProcessInfoFile': 'azureml-logs/process_info.json',
  'ProcessStatusFile': 'azureml-logs/process_status.json'},
 'inputDatasets': [{'dataset': {'id': '3e4d7048-3a64-4687-ad28-a262d98d291b'}, 'consumptionDetails': {'type': 'RunInput', 'inputName': 'input__6228182c', 'mechan

In [13]:
#  the model is saved at path "outputs/001"
# register the model
model = run.register_model(model_name='cifar10torch', model_path='outputs/cifar10torch.pkl')

## Next Steps

1. Learn how to [download model then upload to Azure Storage blobs](../AML-model-download-upload.ipynb)
2. Learn how to [inference using KFServing with model in Azure Storage Blobs](https://aka.ms/kfas)